In [1]:
import os
import sys

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
# Import Data
import pandas as pd
tweets_df = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-influence-env\twitter-influence\data\01_raw\top_accounts_details.csv")
tweets_df['date'] = pd.to_datetime(tweets_df['creation_date'], format='%a %b %d %H:%M:%S +0000 %Y')
tweets_df['month_year'] = tweets_df['date'].apply(lambda x: x.strftime('%Y-%m'))
tweets_df['date_day'] = tweets_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

# calculate per follower metrics 
tweets_df['favorite_count_pf'] = tweets_df['favorite_count']/ tweets_df['follower_count']
tweets_df['retweet_count_pf'] = tweets_df['retweet_count']/ tweets_df['follower_count']
tweets_df['quote_count_pf'] = tweets_df['quote_count']/ tweets_df['follower_count']
tweets_df['reply_count_pf'] = tweets_df['reply_count']/ tweets_df['follower_count']


tweets_df.head()

,tweet_id,creation_date,text,user_id,user_creation_date,username,name,follower_count,following_count,number_of_tweets,...,video_view_count,in_reply_to_status_id,quoted_status_id,date,month_year,date_day,favorite_count_pf,retweet_count_pf,quote_count_pf,reply_count_pf
0,1640814094822383617,Tue Mar 28 20:32:19 +0000 2023,Grateful to all the first responders and volun...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,NaN,NaN,NaN,2023-03-28 20:32:19,2023-03,2023-03-28,0.000048,0.000007,2.330081e-07,0.000002
1,1640485381459025920,Mon Mar 27 22:46:08 +0000 2023,We are failing our children. Guns are now the ...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,2833072.0,NaN,NaN,2023-03-27 22:46:08,2023-03,2023-03-27,0.000917,0.000175,1.643835e-05,0.000127
2,1639039698675937281,Thu Mar 23 23:01:30 +0000 2023,I hope their stories inspire you to help us pr...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,NaN,1.639040e+18,NaN,2023-03-23 23:01:30,2023-03,2023-03-23,0.000023,0.000003,2.029425e-07,0.000004
3,1639039601623851009,Thu Mar 23 23:01:07 +0000 2023,To celebrate the 13th anniversary of the Affor...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,442921.0,NaN,NaN,2023-03-23 23:01:07,2023-03,2023-03-23,0.000095,0.000015,1.142491e-06,0.000006
4,1638956751910215711,Thu Mar 23 17:31:54 +0000 2023,I know so many of you have a story of how acce...,813286,Mon Mar 05 22:08:25 +0000 2007,BarackObama,Barack Obama,133042583,562683,16792,...,NaN,1.638957e+18,NaN,2023-03-23 17:31:54,2023-03,2023-03-23,0.000034,0.000005,2.480409e-07,0.000002


In [3]:
tweets_df['username'].describe()

count            4108
unique            399
top       BBCBreaking
freq              100
Name: username, dtype: object

In [4]:
from scripts.text_processing.sentiment_analysis import SentimentAnalyzer

# sentiment analyser, specify model
analyzer = SentimentAnalyzer('cardiffnlp/twitter-roberta-base-sentiment-latest')

# Get sentiment analysis
tweets_with_sentiment = analyzer.get_sentiment(tweets_df)


C:\Users\johna\anaconda3\envs\twitter-influence-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
tweets_with_sentiment['sentiment_mode'] = tweets_with_sentiment['sentiment']

influence_metrics = tweets_with_sentiment.groupby('user_id').agg(
    {
        'name': [lambda x: x.mode().iloc[0] if not x.mode().empty else None],
        'favorite_count_pf': ['mean'],
        'retweet_count_pf': ['mean'],
        'quote_count_pf': ['mean'],
        'reply_count_pf': ['mean'],
        'tweet_id': ['count'],
        'sentiment': [lambda x: round(x.value_counts(normalize=True), 2).to_dict()], 
        'sentiment_mode': [lambda x: x.mode().iloc[0] if not x.mode().empty else None],
        'follower_count': ['mean'],
        'date_day': [lambda x: x.nunique()]



    }
)

# Calculate the average daily tweets
influence_metrics['avg_daily_tweets'] = influence_metrics[('tweet_id', 'count')] / influence_metrics[('date_day', '<lambda>')]



# Flatten the MultiIndex columns
influence_metrics.columns = ['_'.join(col).strip() for col in influence_metrics.columns.values]

# Reset the index to move the 'username' back to a column
influence_metrics.reset_index(inplace=True)
influence_metrics

,user_id,name_<lambda>,favorite_count_pf_mean,retweet_count_pf_mean,quote_count_pf_mean,reply_count_pf_mean,tweet_id_count,sentiment_<lambda>,sentiment_mode_<lambda>,follower_count_mean,date_day_<lambda>,avg_daily_tweets_
0,428333,CNN Breaking News,0.000009,0.000002,8.513182e-07,2.147219e-06,96,"{'neutral': 0.67, 'negative': 0.31, 'positive'...",neutral,6.412849e+07,27,3.555556
1,759251,CNN,0.000005,0.000001,3.702373e-07,2.345990e-06,99,"{'neutral': 0.68, 'negative': 0.23, 'positive'...",neutral,6.130384e+07,2,49.500000
2,783214,Twitter,0.002210,0.000280,1.554325e-04,1.392290e-04,90,"{'neutral': 0.66, 'positive': 0.27, 'negative'...",neutral,6.434642e+07,62,1.451613
3,807095,The New York Times,0.000007,0.000002,3.581149e-07,8.922909e-07,88,"{'neutral': 0.66, 'negative': 0.18, 'positive'...",neutral,5.499124e+07,2,44.000000
4,813286,Barack Obama,0.026818,0.004638,2.369255e-04,2.609603e-03,93,"{'positive': 0.8, 'neutral': 0.16, 'negative':...",positive,1.235216e+08,54,1.722222
5,2557521,ESPN,0.000173,0.000014,3.316337e-06,3.449520e-06,88,"{'neutral': 0.52, 'positive': 0.48}",neutral,4.547734e+07,5,17.600000
6,5402612,BBC Breaking News,0.000042,0.000007,5.330312e-06,9.118932e-06,100,"{'neutral': 0.59, 'negative': 0.34, 'positive'...",neutral,5.167052e+07,24,4.166667
7,10228272,YouTube,0.002556,0.000612,9.080450e-05,1.485239e-04,97,"{'neutral': 0.62, 'positive': 0.36, 'negative'...",neutral,7.726884e+07,41,2.365854
8,11348282,NASA,0.000083,0.000012,7.848726e-07,2.258591e-06,83,"{'neutral': 0.55, 'positive': 0.45}",neutral,7.064848e+07,26,3.192308
9,14230524,Lady Gaga,5.275835,0.526249,2.616854e-02,1.430575e-01,80,"{'positive': 0.8, 'neutral': 0.2}",positive,6.263672e+07,44,1.818182


In [6]:
print(tweets_df['username'].drop_duplicates())

0           BarackObama
17              bsw5020
23          CivicNation
24         MBK_Alliance
29      ObamaFoundation
             ...       
3991              Drake
4000         Aspiration
4002           caffeine
4007       ajayvirmani1
4015      premierleague
Name: username, Length: 399, dtype: object
